In [730]:
# @title Upload CSV
import pandas as pd
import random

hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')

In [731]:
# @title Gabungkan data_dosen dengan dosen
merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

print("Data Gabungan:")
print(merged_df.head())

total_sks = merged_df['sks'].sum()
print(f"Total slot SKS yang dibutuhkan: {total_sks}")
print(f"Total data dalam merged_df: {len(merged_df)}")

Data Gabungan:
   id_dosen  id_mk_genap kelas                   nama_dosen  \
0         2    211840231     A  Ahmad Azhari, S.Kom., M.Eng   
1         2    211861331     A  Ahmad Azhari, S.Kom., M.Eng   
2         2    211861431     A  Ahmad Azhari, S.Kom., M.Eng   
3         3    211840131     A     Ali Tarmuji, S.T., M.Cs.   
4         3    211860330     A     Ali Tarmuji, S.T., M.Cs.   

                              nama_mk_genap  smt  sks    sifat kategori  \
0                          Grafika Komputer    4    3    Wajib        -   
1                Pengembangan Aplikasi Game    6    3  Pilihan       SC   
2                          Pengenalan Pola     6    3  Pilihan       SC   
3  Analisis dan Perancangan Perangkat Lunak    4    3    Wajib        -   
4                  Rekayasa Perangkat Lunak    6    3    Wajib        -   

    metode  
0  Offline  
1  Offline  
2   Online  
3  Offline  
4  Offline  
Total slot SKS yang dibutuhkan: 282
Total data dalam merged_df: 107


Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [732]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
    wolf = []
    for id_counter, (hari, ruang, jam) in enumerate(
        [(h, r, j) for h in hari_list for r in ruang_list for j in jam_list], start=1
    ):
        wolf.append({
            "id_slot": id_counter,
            "mata_kuliah": None,
            "dosen": None,
            "ruang": ruang,
            "hari": hari,
            "jam_mulai": jam['jam_awal'],
            "jam_selesai": jam['jam_akhir'],
            "kelas": None,
            "sks": None,
            "metode": None
        })
    return wolf

slots = slot_generator()
for slot in slots[:5]:
    print(slot)

print("\n")
for slot in slots[-5:]:
    print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}


{'id_slot': 500, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.63', 'hari'

In [733]:
# @title Inisialisasi Populasi (GWO)
from collections import defaultdict
from datetime import datetime

def time_to_minutes(t):
    try:
        return datetime.strptime(t, "%H:%M:%S").hour * 60 + datetime.strptime(t, "%H:%M:%S").minute
    except ValueError:
        return datetime.strptime(t, "%H:%M").hour * 60 + datetime.strptime(t, "%H:%M").minute

def create_random_schedule():
    slots = slot_generator()
    merged_shuffled = merged_df.sample(frac=1).iterrows()
    
    # Struktur tracking untuk hard constraints
    room_schedule = defaultdict(list)
    teacher_schedule = defaultdict(list)

    for index, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']

        possible_positions = list(range(len(slots) - sks + 1))
        random.shuffle(possible_positions)

        placed = False
        for i in possible_positions:
            block = slots[i:i+sks]
            
            # Validasi dasar slot
            all_empty = all(slot['mata_kuliah'] is None for slot in block)
            same_hari = len({slot['hari'] for slot in block}) == 1
            same_ruang = (metode == 'Online') or (len({slot['ruang'] for slot in block}) == 1)
            
            if not (all_empty and same_ruang and same_hari):
                continue
                
            # Ekstrak informasi waktu
            target_hari = block[0]['hari']
            target_ruang = block[0]['ruang'] if metode != 'Online' else None
            block_start = time_to_minutes(block[0]['jam_mulai'])
            block_end = time_to_minutes(block[-1]['jam_selesai'])
            
            # Hard constraint 1: Cek konflik ruang
            room_conflict = False
            if metode != 'Online':
                for existing_start, existing_end in room_schedule.get((target_ruang, target_hari), []):
                    if not (block_end <= existing_start or block_start >= existing_end):
                        room_conflict = True
                        break
                    
            # Hard constraint 2: Cek konflik dosen
            teacher_conflict = False
            for existing_start, existing_end in teacher_schedule.get((dosen, target_hari), []):
                if not (block_end <= existing_start or block_start >= existing_end):
                    teacher_conflict = True
                    break
            
            if not room_conflict and not teacher_conflict:
                # Update data slot
                ruang = 'Online' if metode == 'Online' else target_ruang
                for slot in block:
                    slot.update({
                        "mata_kuliah": mata_kuliah,
                        "dosen": dosen,
                        "kelas": kelas,
                        "sks": sks,
                        "metode": metode,
                        "ruang": ruang
                    })
                # Update tracking
                if metode != 'Online':
                    room_schedule[(target_ruang, target_hari)].append((block_start, block_end))
                teacher_schedule[(dosen, target_hari)].append((block_start, block_end))
                placed = True
                break
                
                # Fallback untuk soft constraint jika tidak ditemukan slot
        if not placed:
            # Implementasi fallback untuk soft constraint di sini
            pass
    return slots

population_size = 5
population = [create_random_schedule() for _ in range(population_size)]

for i in range(population_size):
    print(f"\nIndividu {i+1}:")
    for slot in population[i][:5]:
    # for slot in population[i]:
        print(slot)


Individu 1:
{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': 'Praktikum Algoritma Pemrograman', 'dosen': 'Ardiansyah, Dr., S.T., M.Cs.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': 'A', 'sks': 1, 'metode': 'Offline'}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}

Individu 2:


Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
jadwal dosen mengajar kelas yang berbeda pada jam yang sama
jadwal ruang kelas ruang kelas digunakan lebih dari 1 dosen 

Konstrain ringan:
dosen mengajar pada hari yang sama
kelas dijadwalkan pada waktu paling akhir dan paling awal pada hari itu

In [734]:
# @title Fitness Function (GWO)
from collections import defaultdict

def calculate_fitness(schedule):
    max_score = 100
    penalty = 0
    
    # Struktur tracking untuk semua konstrain
    teacher_schedule = defaultdict(list)
    room_schedule = defaultdict(list)
    teacher_daily_courses = defaultdict(set)
    class_time_bounds = defaultdict(lambda: {'min': float('inf'), 'max': -float('inf')})
    penalized_classes = set()

    # Fungsi untuk menghitung overlap
    def count_overlaps(intervals):
        intervals.sort()
        overlaps = 0
        for i in range(1, len(intervals)):
            if intervals[i][0] < intervals[i-1][1]:
                overlaps += 1
        return overlaps

    # First pass: Kumpulkan semua data
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
            
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        hari = slot['hari']
        dosen = slot['dosen']
        ruang = slot['ruang']
        kelas = slot['kelas']
        mk = slot['mata_kuliah']

        # Track untuk hard constraints
        teacher_schedule[(dosen, hari)].append((start, end))
        if ruang != 'Online':
            room_schedule[(ruang, hari)].append((start, end))

        # Track untuk soft constraints
        teacher_daily_courses[(dosen, hari)].add(mk)
        class_time_bounds[(kelas, hari)]['min'] = min(class_time_bounds[(kelas, hari)]['min'], start)
        class_time_bounds[(kelas, hari)]['max'] = max(class_time_bounds[(kelas, hari)]['max'], start)

    # Hitung hard constraints
    # 1. Dosen mengajar di kelas berbeda pada jam yang sama
    for intervals in teacher_schedule.values():
        penalty += count_overlaps(intervals) * 1  # Berat 1 per konflik
        
    # 2. Ruang digunakan oleh lebih dari 1 kelas di waktu sama
    for intervals in room_schedule.values():
        penalty += count_overlaps(intervals) * 1  # Berat 1 per konflik

    # Hitung soft constraints
    # 1. Dosen mengajar di hari yang sama
    for courses in teacher_daily_courses.values():
        if len(courses) > 1:
            penalty += 0.5 * (len(courses) - 1)  # Ringan 0.5 per tambahan MK
            
    # 2. Kelas di jam pertama/terakhir
    for (kelas, hari), bounds in class_time_bounds.items():
        if (kelas, hari) not in penalized_classes:
            min_time = bounds['min']
            max_time = bounds['max']
            
            # Cek keberadaan slot di jam awal/akhir
            has_min = any(
                time_to_minutes(slot['jam_mulai']) == min_time 
                for slot in schedule 
                if slot['kelas'] == kelas 
                and slot['hari'] == hari 
                and slot['mata_kuliah']
            )
            
            has_max = any(
                time_to_minutes(slot['jam_mulai']) == max_time 
                for slot in schedule 
                if slot['kelas'] == kelas 
                and slot['hari'] == hari 
                and slot['mata_kuliah']
            )
            
            if has_min:
                penalty += 0.5
            if has_max:
                penalty += 0.5
                
            penalized_classes.add((kelas, hari))

    return {
        'score': max(0, max_score - penalty),  # Pastikan tidak negatif
        'penalty': penalty,
        'violations': {
            'hard_teacher': sum(count_overlaps(v) for v in teacher_schedule.values()),
            'hard_room': sum(count_overlaps(v) for v in room_schedule.values()),
            'soft_teaching_day': sum(len(v)-1 for v in teacher_daily_courses.values() if len(v) > 1),
            'soft_edge_time': sum(1 for _ in penalized_classes)
        }
    }

# Generate populasi
population_size = 5
population = [create_random_schedule() for _ in range(population_size)]

for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    print(f"Fitness: {fitness['score']}")
    print(f"Total Penalty: {fitness['penalty']}")
    print("Detail Pelanggaran:")
    print(f"- Konflik Dosen: {fitness['violations']['hard_teacher']}")
    print(f"- Konflik Ruang: {fitness['violations']['hard_room']}")
    print(f"- Dosen mengajar lebih dari 1 MK per hari: {fitness['violations']['soft_teaching_day']}")
    print(f"- Kelas Jam Awal Hari/Akhir Hari: {fitness['violations']['soft_edge_time']}")


Individu 1:
Fitness: 59.5
Total Penalty: 40.5
Detail Pelanggaran:
- Konflik Dosen: 0
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 11
- Kelas Jam Awal Hari/Akhir Hari: 35

Individu 2:
Fitness: 58.5
Total Penalty: 41.5
Detail Pelanggaran:
- Konflik Dosen: 1
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 19
- Kelas Jam Awal Hari/Akhir Hari: 31

Individu 3:
Fitness: 54.5
Total Penalty: 45.5
Detail Pelanggaran:
- Konflik Dosen: 3
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 17
- Kelas Jam Awal Hari/Akhir Hari: 34

Individu 4:
Fitness: 54.0
Total Penalty: 46.0
Detail Pelanggaran:
- Konflik Dosen: 2
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 22
- Kelas Jam Awal Hari/Akhir Hari: 33

Individu 5:
Fitness: 58.0
Total Penalty: 42.0
Detail Pelanggaran:
- Konflik Dosen: 1
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 16
- Kelas Jam Awal Hari/Akhir Hari: 33


In [ ]:
# @title Final Improved GWO Implementation
import copy
import random
from collections import defaultdict
from datetime import datetime

def time_to_minutes(t):
    try:
        return datetime.strptime(t, "%H:%M:%S").hour * 60 + datetime.strptime(t, "%H:%M:%S").minute
    except ValueError:
        return datetime.strptime(t, "%H:%M").hour * 60 + datetime.strptime(t, "%H:%M").minute

def minutes_to_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    return f"{hours:02d}:{mins:02d}:00"

def adjust_slot_times(schedule):
    adjusted = copy.deepcopy(schedule)
    course_blocks = defaultdict(list)
    
    # Kelompokkan berdasarkan semua atribut penting
    for idx, slot in enumerate(adjusted):
        if slot['mata_kuliah']:
            key = (slot['mata_kuliah'], slot['kelas'], slot['dosen'], slot['hari'])
            course_blocks[key].append((idx, slot))
    
    for key, slot_data in course_blocks.items():
        slots = [s for idx, s in slot_data]
        sks = slots[0]['sks']
        
        if len(slots) != sks:
            continue
            
        # Hitung total durasi berdasarkan SKS (1 SKS = 50 menit)
        total_duration = sks * 50
        start_time = time_to_minutes(slots[0]['jam_mulai'])
        end_time = start_time + total_duration
        
        # Update semua slot dalam blok
        start_str = minutes_to_time(start_time)
        end_str = minutes_to_time(end_time)
        for idx, slot in slot_data:
            adjusted[idx]['jam_mulai'] = start_str
            adjusted[idx]['jam_selesai'] = end_str
            
    return adjusted

def repair_schedule(schedule):
    repaired = adjust_slot_times(schedule)
    teacher_map = defaultdict(list)
    room_map = defaultdict(list)
    
    # Proses slot secara acak untuk diversifikasi
    sorted_slots = [s for s in repaired if s['mata_kuliah']]
    random.shuffle(sorted_slots)
    
    new_schedule = [s.copy() for s in repaired]
    
    for slot in sorted_slots:
        original_idx = next(
            (i for i, s in enumerate(new_schedule) 
             if s['id_slot'] == slot['id_slot']), None
        )
        
        if original_idx is None:
            continue
            
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        conflict = False
        
        # Cek konflik dosen
        for interval in teacher_map[(slot['dosen'], slot['hari'])]:
            if interval['start'] < end and interval['end'] > start:
                conflict = True
                break
                
        # Cek konflik ruang
        if not conflict and slot['ruang'] != 'Online':
            for interval in room_map[(slot['ruang'], slot['hari'])]:
                if interval['start'] < end and interval['end'] > start:
                    conflict = True
                    break
        
        if conflict:
            # Cari slot kosong dengan durasi yang cukup
            required_duration = end - start
            for i, candidate in enumerate(new_schedule):
                if not candidate['mata_kuliah']:
                    candidate_start = time_to_minutes(candidate['jam_mulai'])
                    candidate_end = time_to_minutes(candidate['jam_selesai'])
                    if (candidate_end - candidate_start) >= required_duration:
                        new_schedule[i] = {
                            **candidate,
                            'mata_kuliah': slot['mata_kuliah'],
                            'dosen': slot['dosen'],
                            'kelas': slot['kelas'],
                            'sks': slot['sks'],
                            'metode': slot['metode'],
                            'ruang': slot['ruang']
                        }
                        break
            else:
                new_schedule[original_idx]['mata_kuliah'] = None
        else:
            teacher_map[(slot['dosen'], slot['hari'])].append({'start': start, 'end': end})
            if slot['ruang'] != 'Online':
                room_map[(slot['ruang'], slot['hari'])].append({'start': start, 'end': end})
    
    return adjust_slot_times(new_schedule)

def update_wolf_position(wolf, alpha, beta, delta, a):
    new_wolf = copy.deepcopy(wolf)
    wolf_slots = {s['id_slot']: s for s in new_wolf}
    
    # Parameter adaptif
    exploration_rate = 0.5 * a
    
    for leader in [alpha, beta, delta]:
        for slot in leader:
            if not slot['mata_kuliah']:
                continue
                
            current_slot = wolf_slots.get(slot['id_slot'], None)
            if current_slot:
                # Hitung probabilitas crossover
                if random.random() < exploration_rate:
                    # Eksplorasi: ambil dari leader
                    wolf_slots[slot['id_slot']] = copy.deepcopy(slot)
                else:
                    # Eksploitasi: pertahankan solusi terbaik
                    current_fitness = calculate_fitness([current_slot])['score']
                    leader_fitness = calculate_fitness([slot])['score']
                    if leader_fitness > current_fitness:
                        wolf_slots[slot['id_slot']] = copy.deepcopy(slot)
    
    return list(wolf_slots.values())

def gwo_optimize(population, max_iter=50):
    a = 2.0
    best_scores = []
    
    def get_leaders(pop):
        evaluated = [(s, calculate_fitness(s)) for s in pop]
        sorted_pop = sorted(evaluated, key=lambda x: x[1]['score'], reverse=True)
        return [s[0] for s in sorted_pop[:3]], [s[1]['score'] for s in sorted_pop[:3]]
    
    for iter in range(max_iter):
        leaders, scores = get_leaders(population)
        alpha, beta, delta = leaders
        a = 2.0 - (2.0 * iter)/max_iter
        
        new_population = []
        for wolf in population:
            if wolf in leaders:
                new_population.append(copy.deepcopy(wolf))
                continue
                
            new_wolf = update_wolf_position(wolf, alpha, beta, delta, a)
            new_wolf = repair_schedule(new_wolf)
            new_population.append(new_wolf)
        
        # Elitism + random immigrants
        population = new_population[:80] + [create_random_schedule() for _ in range(20)]
        best_scores.append(scores[0])
        print(f"Iter {iter+1} - Best: {scores[0]} | a={a:.2f}")
    
    return get_leaders(population)[0][0], best_scores

# Fungsi Output dengan Validasi SKS
def print_schedule(schedule):
    printed = set()
    for slot in schedule:
        if slot['mata_kuliah']:
            key = (slot['mata_kuliah'], slot['kelas'], slot['dosen'], slot['hari'])
            if key not in printed:
                # Validasi durasi sesuai SKS
                duration = time_to_minutes(slot['jam_selesai']) - time_to_minutes(slot['jam_mulai'])
                expected_duration = slot['sks'] * 50  # 1 SKS = 50 menit
                
                if duration != expected_duration:
                    print(f"! Warning: Invalid duration for {slot['mata_kuliah']} ({duration} vs {expected_duration} minutes)")
                
                start = datetime.strptime(slot['jam_mulai'], "%H:%M:%S").strftime("%H:%M")
                end = datetime.strptime(slot['jam_selesai'], "%H:%M:%S").strftime("%H:%M")
                print(f"{slot['hari']} {start}-{end} ({slot['sks']} SKS) | {slot['ruang']} | {slot['mata_kuliah']} ({slot['dosen']})")
                printed.add(key)

# Eksekusi
best_schedule, _ = gwo_optimize(population, max_iter=20)
print("\n=== Jadwal Terbaik ===")
print_schedule(best_schedule)

Iter 1 - Best: 59.5 | a=2.00
Iter 2 - Best: 62.0 | a=1.90
Iter 3 - Best: 62.0 | a=1.80
Iter 4 - Best: 62.5 | a=1.70
Iter 5 - Best: 62.5 | a=1.60
Iter 6 - Best: 62.5 | a=1.50
